In [1]:
import os
import time
import pickle
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [23]:
video_dir = '/mnt/disks/datastorage/videos/'
weights_dir = '/mnt/disks/datastorage/weights/'
# file = 'v_Shooting_g09_v1_a.mov'
#file = 'v_Shooting_g18_v1_c.MP4'
#file = 'v_Shooting_g11_v1_b.mov'
file = 'v_Misc_g29_v1_b.mov'

video_path = os.path.join(video_dir, 'ballroom', file)
nb_classes = 3
skip_num = 1
svm_model = '/home/mlp/two-stream-action-recognition/demo_svm.pkl'
prediction_path = '/mnt/disks/datastorage/videos/'
flownet_weights = os.path.join(weights_dir, 'FlowNet2-CSS.pth.tar')
spatial_weights = os.path.join('/home/mlp/two-stream-action-recognition/record/spatial', 'model_best_DEMO.pth.tar')
motion_weights = os.path.join('/home/mlp/two-stream-action-recognition/record/motion', 'model_best_DEMO.pth.tar')
flownet_model = 'FlowNet2CSS'

In [25]:
!python3 -W ignore pipeline.py --stream $video_path \
                               --nb_classes $nb_classes \
                               --skip_frames $skip_num \
                               --svm $svm_model \
                               --save $prediction_path \
                               --model $flownet_model \
                               --optical_weights $flownet_weights \
                               --spatial_weights $spatial_weights \
                               --motion_weights $motion_weights

Parsing Arguments
  [0.242s] fp16: False
  [0.242s] fp16_scale: 1024.0
  [0.242s] image_size: [224, 224]
  [0.242s] inference_size: [-1, -1]
  [0.242s] loss: L1Loss
  [0.242s] model: FlowNet2CSS
  [0.242s] model_batchNorm: False
  [0.243s] model_div_flow: 20.0
  [0.243s] motion_weights: /home/mlp/two-stream-action-recognition/record/motion/model_best_DEMO.pth.tar
  [0.243s] nb_classes: 3
  [0.243s] number_gpus: 1
  [0.243s] optical_weights: /mnt/disks/datastorage/weights/FlowNet2-CSS.pth.tar
  [0.243s] rgb_max: 255.0
  [0.243s] save: /mnt/disks/datastorage/videos/
  [0.243s] seed: 1
  [0.243s] skip_frames: 1
  [0.243s] spatial_weights: /home/mlp/two-stream-action-recognition/record/spatial/model_best_DEMO.pth.tar
  [0.243s] stream: /mnt/disks/datastorage/videos/ballroom/v_Misc_g29_v1_b.mov
  [0.243s] svm: /home/mlp/two-stream-action-recognition/fens_svm.pkl
  [0.243s] Operation finished

Building FlowNet2CSS model
  [3.440s] Number of parameters: 116565942
  [3.440s] Initializing CUDA


In [19]:
prediction_files = sorted(glob.glob(os.path.join(prediction_path, 'predictions_*.pkl')), reverse=True)

with open(prediction_files[0], 'rb') as pf:
    predictions = pickle.load(pf)
    predictions = np.array(predictions).squeeze()
    actions = np.argmax(predictions, axis=1) 

In [20]:
action_dict = {
    0: 'Theft',
    1: 'Assault',
    2: 'Shooting',
    3: 'No Action',
}

cap = cv2.VideoCapture(video_path)

frame_time = 0
frame_idx = 0
display = None
i = 0
while True:
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break
    
    try:
        label = action_dict[actions[frame_idx]]
    except IndexError:
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.putText(
        frame,
        label,
        (10, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (255, 255, 255),
        thickness=3
    )
    
    if i < 0:
        frame_time += time.time() - start_time
        frame_idx += 1
        i+=1
        continue
        
        
    else:
        plt.axis('off')
        plt.imshow(frame)
        plt.show()
        clear_output(wait=True)

        frame_time += time.time() - start_time
        frame_idx += 1
cap.release()

frame_time /= frame_idx
print('Avg. frame processing time: {} s'.format(round(frame_time, 2)))

KeyboardInterrupt: 